In [58]:
import os
import numpy as np
from PIL import Image
import PIL.ImageOps    
from astropy.io import fits
from astropy.table import Table

import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
import image_registration
from astropy.nddata import Cutout2D
from astropy import units as u
import matplotlib.image as mpimg

from __future__ import print_function
#import cv2
import numpy as np

#from photutils import DAOStarFinder
#from photutils import CircularAperture
#from photutils import aperture_photometry

from astropy.stats import sigma_clipped_stats

from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

 
import pandas as pd
import pandasql as ps
plt.style.use(astropy_mpl_style)

# from regions.core import PixCoord
# from regions.shapes.circle import CirclePixelRegion
import scipy.misc
import time

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


In [57]:
def makeFits(inName,imageExtension='.jpg'):
    image = Image.open(inName)
    xsize, ysize = image.size
    print("Image size: {} x {}".format(xsize, ysize))
    # plt.imshow(image)

    r, g, b = image.split()
    r_data = np.array(r.getdata()) # data is now an array of length ysize*xsize
    g_data = np.array(g.getdata())
    b_data = np.array(b.getdata())
    print(r_data.shape)

    r_data = r_data.reshape(ysize, xsize)[::-1]
    g_data = g_data.reshape(ysize, xsize)[::-1]
    b_data = b_data.reshape(ysize, xsize)[::-1]
    
    print(inName)
    inName = inName.replace(imageExtension,'')
    print(inName)
    try:
        os.remove(inName+"red.fits")
        os.remove(inName+"green.fits")
        os.remove(inName+"blue.fits")
    except:
        print("Can't remove as the file doesn't exist")

    red = fits.PrimaryHDU(r_data)
    hdur = fits.HDUList([red])
    hdur.writeto(inName+'red.fits')
    
    green = fits.PrimaryHDU(g_data)
    hdug = fits.HDUList([green])
    hdug.writeto(inName+'green.fits')

    blue = fits.PrimaryHDU(b_data)
    hdub = fits.HDUList([blue])
    hdub.writeto(inName+'blue.fits')

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [15]:
def saveImData(imData,name):
    im = fits.PrimaryHDU(imData)
    hdub = fits.HDUList([im])
    hdub.writeto(name)

In [22]:
def getImageData(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [63]:
def backgroundSubtraction(imData):
    for x in range(len(imData)):
        meanVal = np.mean(imData[x])
        meanLimit = meanVal*0.75
        background = imData[x][imData[x]<meanLimit]
        backGroundVal = np.mean(background) 
        imData[x] = imData[x] - backGroundVal
        imData[imData < 0] = 0
    return imData

In [29]:
def makeAverageImage(redimage,blueimage,greenimage):
    moleImage = (redimage+blueimage+greenimage)/3
    return moleImage


In [30]:
im = '/home/mj1e16/moleGazer/Photos/F/29018Invertedblue.fits'
loc1 = im.index('.fits')
print(im[:loc1])

/home/mj1e16/moleGazer/Photos/F/29018Invertedblue


In [66]:
def combineMakeImage(imageName,inDirectory='',outPutDirectory=''):
    imShort = imageName.replace('.jpg','')
    
    fitsIm = imageName.replace('.jpg','.fits')
    imagedata = Image.open(inDirectory+imageName)
    inverted_image = PIL.ImageOps.invert(imagedata)
    inverted_image.save(inDirectory+imShort+'NewInverted.jpg')
    print(inDirectory+imShort+'NewInverted.jpg')
    makeFits(inDirectory+imShort+'NewInverted.jpg')
    imGreen,imBlue,imRed = inDirectory+imShort+'NewInvertedgreen.fits',inDirectory+imShort+'NewInvertedblue.fits',inDirectory+imShort+'NewInvertedred.fits'
    
    redIm = getImageData(imRed,0)
    blueIm = getImageData(imBlue,0)
    greenIm = getImageData(imGreen,0)
    avIm = makeAverageImage(redIm,blueIm,greenIm)
    bsAvIm = avIm #backgroundSubtraction(avIm)
    try:
        os.remove(outPutDirectory+imShort+'InvertedAverage.fits')
    except:
        print("Can't remove {} as the file doesn't exist".format(outPutDirectory+imShort+'InvertedAverage.fits'))
    saveImData(bsAvIm,outPutDirectory+imShort+'InvertedAverage.fits')

In [60]:
print(inputFinal[4])

79941.jpg


In [ ]:
imdata = getImageData

In [67]:
combineMakeImage(inputFinal[4],inDirectory='/home/mj1e16/moleGazer/Photos/F/',outPutDirectory='/home/mj1e16/moleGazer/Photos/F/fitsImages/')

/home/mj1e16/moleGazer/Photos/F/79941NewInverted.jpg
Image size: 2477 x 3800
(9412600,)
/home/mj1e16/moleGazer/Photos/F/79941NewInverted.jpg
/home/mj1e16/moleGazer/Photos/F/79941NewInverted


In [53]:
inputImages = os.listdir('/home/mj1e16/moleGazer/Photos/F/')
inputRefined = [x for x in inputImages if 'Inverted' not in x]
inputFinal = [x for x in inputRefined if '.jpg' in x]
inputFinal = [x for x in inputFinal if x !='aligned.jpg']

In [54]:
inputFinal

['29007.jpg',
 '79945.jpg',
 '29006.jpg',
 '29011.jpg',
 '79941.jpg',
 '29008.jpg',
 '29014.jpg',
 '79940.jpg',
 '29013.jpg',
 '79947.jpg',
 '29024.jpg',
 '29015.jpg',
 '79946.jpg',
 '29023.jpg',
 '79949.jpg',
 '29017.jpg',
 '29019.jpg',
 '29022.jpg',
 '29012.jpg',
 '79948.jpg',
 '79942.jpg',
 '29010.jpg',
 '79943.jpg',
 '29005.jpg',
 '29016.jpg',
 '29009.jpg',
 '29021.jpg',
 '79952.jpg',
 '29020.jpg',
 '79953.jpg',
 '79944.jpg',
 '79951.jpg',
 '79950.jpg',
 '29018.jpg']

In [68]:
for x in inputFinal:
    combineMakeImage(x,inDirectory='/home/mj1e16/moleGazer/Photos/F/',outPutDirectory='/home/mj1e16/moleGazer/Photos/F/fitsImages/')

/home/mj1e16/moleGazer/Photos/F/29007NewInverted.jpg
Image size: 3800 x 2599
(9876200,)
/home/mj1e16/moleGazer/Photos/F/29007NewInverted.jpg
/home/mj1e16/moleGazer/Photos/F/29007NewInverted
/home/mj1e16/moleGazer/Photos/F/79945NewInverted.jpg
Image size: 2477 x 3800
(9412600,)
/home/mj1e16/moleGazer/Photos/F/79945NewInverted.jpg
/home/mj1e16/moleGazer/Photos/F/79945NewInverted
/home/mj1e16/moleGazer/Photos/F/29006NewInverted.jpg
Image size: 3800 x 2609
(9914200,)
/home/mj1e16/moleGazer/Photos/F/29006NewInverted.jpg
/home/mj1e16/moleGazer/Photos/F/29006NewInverted
/home/mj1e16/moleGazer/Photos/F/29011NewInverted.jpg
Image size: 2490 x 3800
(9462000,)
/home/mj1e16/moleGazer/Photos/F/29011NewInverted.jpg
/home/mj1e16/moleGazer/Photos/F/29011NewInverted
/home/mj1e16/moleGazer/Photos/F/79941NewInverted.jpg
Image size: 2477 x 3800
(9412600,)
/home/mj1e16/moleGazer/Photos/F/79941NewInverted.jpg
/home/mj1e16/moleGazer/Photos/F/79941NewInverted
/home/mj1e16/moleGazer/Photos/F/29008NewInverted.j

In [10]:
redIm = getImageData('/home/mj1e16/moleGazer/Photos/F/29018Invertedblue.fits',0)
blueIm = getImageData('/home/mj1e16/moleGazer/Photos/F/29018Invertedblue.fits',0)
greenIm = getImageData('/home/mj1e16/moleGazer/Photos/F/29018Invertedgreen.fits',0)
avIm = makeAverageImage(redIm,blueIm,greenIm)
bsAvIm = backgroundSubtraction(avIm)

KeyboardInterrupt: 

In [ ]:
saveImData(bsAvIm,'/home/mj1e16/moleGazer/Photos/F/29018moleAv.fits')

In [50]:
redIm = getImageData('/home/mj1e16/moleGazer/Photos/F/79948Invertedred.fits',0)
blueIm = getImageData('/home/mj1e16/moleGazer/Photos/F/79948Invertedblue.fits',0)
greenIm = getImageData('/home/mj1e16/moleGazer/Photos/F/79948Invertedgreen.fits',0)

In [51]:
moleBrightIm = makeMoleBrightImage(redIm,blueIm,greenIm)
bsMoleIm = backgroundSubtraction(moleBrightIm) 

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in divide
  from ipykernel import kernelapp as app


In [80]:
redblue = redIm*blueIm
greenSquared = greenIm
moleTestIm = np.true_divide(redblue,greenSquared)
meanImage = (redIm+greenIm+blueIm)/3
molebrightNew = moleTestIm

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [81]:
saveImData(molebrightNew,'/home/mj1e16/moleGazer/Photos/F/799moleBrightNormal3.fits')

In [82]:
meanImage = (redIm+greenIm+blueIm)/3
saveImData(meanImage,'/home/mj1e16/moleGazer/Photos/F/799Mean.fits')

In [83]:
testImageData = backgroundSubtraction(meanImage)

In [86]:
saveImData(testImageData,'/home/mj1e16/moleGazer/Photos/F/799MeanBS.fits')